### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


In [1]:
# Importo librerias
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score


from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pandas as pd

In [2]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [3]:
#veo el primer documento
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [4]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [5]:
#Reviso la distribución del target
df_tgt = pd.DataFrame(newsgroups_test.target, columns=['target'])

df_tgt['target'].value_counts(dropna=False, normalize =True).sort_index()

,proportion
target,
0,0.042353
1,0.051646
2,0.052310
3,0.052045
4,0.051115
5,0.052443
6,0.051779
7,0.052576
8,0.052841


In [6]:
# instancio varios vectorizadores

tfidfvect = TfidfVectorizer()
tfidfvect_m = TfidfVectorizer(min_df=2) # min_df: 2. para matener un término dentro del vocabulario debe aparecer al menos 2 veces
countvect = CountVectorizer()

In [7]:
# fit sobre train del vectorizador TfidfVectorizer
X_train_tfidf = tfidfvect.fit_transform(newsgroups_train.data)

print(type(X_train_tfidf))
print(f'shape: {X_train_tfidf.shape}')
print(f'cantidad de documentos: {X_train_tfidf.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train_tfidf.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# fit sobre train del vectorizador TfidfVectorizer con min_df = 2
X_train_tfidf_m = tfidfvect_m.fit_transform(newsgroups_train.data)

print(type(X_train_tfidf_m))
print(f'shape: {X_train_tfidf_m.shape}')
print(f'cantidad de documentos: {X_train_tfidf_m.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train_tfidf_m.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 39423)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 39423


In [9]:
# fit sobre train del vectorizador CountVectorizer
X_train_count = countvect.fit_transform(newsgroups_train.data)

print(type(X_train_count))
print(f'shape: {X_train_count.shape}')
print(f'cantidad de documentos: {X_train_count.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train_count.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [10]:
y_train = newsgroups_train.target

**Punto 1**

In [11]:
idx = [2000]
X_train = X_train_tfidf

cossim = cosine_similarity(X_train[idx], X_train)[0]

# veo el idx del documento y los 5 más similares
mostsim = np.argsort(cossim)[::-1][0:6]

# veo las clases:
for i in mostsim:
  print('idx:',i)
  print(newsgroups_train.target_names[y_train[i]])

idx: 2000
rec.motorcycles
idx: 4271
talk.politics.misc
idx: 9623
talk.politics.mideast
idx: 4253
talk.politics.misc
idx: 3596
talk.politics.misc
idx: 10229
talk.religion.misc


El documento 2000 corresponde a la clase 'rec.motorcycles' y ninguno de los documentos similares comparten esa clase. Los documentos que resultan ser similares pero con que tienen una clase distinta, se da porque los documentos comparten vocabulario con la clase del documento original.
Además es notorio que los documentos sobre política tienen una extensión mayor y esto aumenta la chance de compartir términos con documentos de otras clases.

In [12]:
print(newsgroups_train.data[2000])



_Cycle World_ puts one out, but I'm sure it's not very objective.  Try talking
with dealers and the people that hang out there, as well as us.  We love to
give advice.


Most of the bigger banks have a blue book which includes motos -- ask for the
one with RVs in it.


Couldn't help you here.


You're reading it.

----------------------------------------------------------------------------


In [13]:
print(newsgroups_train.data[4271])

THE WHITE HOUSE

                    Office of the Press Secretary
______________________________________________________________
For Immediate Release                             April 13, 1993     

	     
                      REMARKS BY THE PRESIDENT,
               SECRETARY OF EDUCATION RICHARD RILEY AND
                   SECRETARY OF LABOR ROBERT REICH  IN 
                GOALS 2000 SATELLITE TOWN HALL MEETING
	     
                     Chamber of Commerce Building
                           Washington, D.C.   



8:30 P.M. EDT
	     
	     
	     SECRETARY RILEY:  Good evening and welcome to all of you 
in the thousands of communities around the country that are taking 
part in this satellite town meeting for the month of April.
	     
	     You know, today is April 13th.  In 1743, Thomas 
Jefferson was born, 250 years ago.  I think that's appropriate to 
mention at the beginning of this meeting because since that time he 
has been, of course, a person who has been one that 

In [14]:
idx = [8000]
X_train = X_train_tfidf

cossim = cosine_similarity(X_train[idx], X_train)[0]

# veo el idx del documento y los 5 más similares
mostsim = np.argsort(cossim)[::-1][0:6]

# veo las clases:
for i in mostsim:
  print('idx:',i)
  print(newsgroups_train.target_names[y_train[i]])

idx: 8000
talk.politics.misc
idx: 6552
talk.religion.misc
idx: 9623
talk.politics.mideast
idx: 2240
alt.atheism
idx: 6894
talk.politics.guns
idx: 3902
talk.religion.misc


El documento 8000 corresponde a la clase 'talk.politics.misc' y dentro de los 5 documentos similares solo uno corresponde a esta misma clase. El resto de los documentos tiene una clase distinta y esto se debe a que los documentos comparten vocabulario con la clase del documento original.

In [15]:
idx = [6800]
X_train = X_train_tfidf

cossim = cosine_similarity(X_train[idx], X_train)[0]

# veo el idx del documento y los 5 más similares
mostsim = np.argsort(cossim)[::-1][0:6]

# veo las clases:
for i in mostsim:
  print('idx:',i)
  print(newsgroups_train.target_names[y_train[i]])

idx: 6800
talk.politics.misc
idx: 4435
talk.politics.misc
idx: 3061
talk.politics.misc
idx: 5856
sci.crypt
idx: 3746
talk.politics.misc
idx: 4253
talk.politics.misc


El documento 6800 corresponde a la clase 'talk.politics.misc' y 4 de los documentos más similares también. Solo 1 de los documentos tiene una clase distinta que es 'sci.crypt' y esto se debe a que el documento comparte vocabulario con los documentos de la clase 'talk.politics.misc'.

In [16]:
idx = [4300]
X_train = X_train_tfidf

cossim = cosine_similarity(X_train[idx], X_train)[0]

# veo el idx del documento y los 5 más similares
mostsim = np.argsort(cossim)[::-1][0:6]

# veo las clases:
for i in mostsim:
  print('idx:',i)
  print(newsgroups_train.target_names[y_train[i]])

idx: 4300
sci.crypt
idx: 6243
sci.crypt
idx: 10575
sci.crypt
idx: 2825
sci.crypt
idx: 2256
sci.crypt
idx: 11008
sci.crypt


El documento 4300 corresponde a la clase 'sci.crypt' y los 5 documentos más similares también.

In [17]:
idx = [1300]
X_train = X_train_tfidf

cossim = cosine_similarity(X_train[idx], X_train)[0]

# veo el idx del documento y los 5 más similares
mostsim = np.argsort(cossim)[::-1][0:6]

# veo las clases:
for i in mostsim:
  print('idx:',i)
  print(newsgroups_train.target_names[y_train[i]])

idx: 1300
sci.med
idx: 8550
sci.med
idx: 8660
sci.med
idx: 2189
sci.med
idx: 3808
sci.med
idx: 3652
sci.med


El documento 1300 corresponde a la clase 'sci.med' y los 5 documentos más similares también.

**Punto 2**

In [18]:
#Entreno un modelo MultinomialNB sobre el dataset de train vectorizado con TfidfVectorizer con los parámetros por default.

#fit del modelo sobre train
clf = MultinomialNB()
clf.fit(X_train_tfidf, y_train)

#aplico el transform del vectorizador fitteado en train
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

#calculo el f1_score macro
f1_1 = f1_score(y_test, y_pred, average='macro').item()
print(f1_1)

0.5854345727938506


In [19]:
#Entreno un modelo MultinomialNB sobre el dataset de train vectorizado con TfidfVectorizer con min_df = 2.

#fit del modelo sobre train
clf = MultinomialNB()
clf.fit(X_train_tfidf_m, y_train)

#aplico el transform del vectorizador fitteado en train
X_test = tfidfvect_m.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

#calculo el f1_score macro
f1_2 = f1_score(y_test, y_pred, average='macro').item()
print(f1_2)

0.5970494639319617


In [20]:
#Entreno un modelo MultinomialNB sobre el dataset de train vectorizado con CountVectorizer

#fit del modelo sobre train
clf = MultinomialNB()
clf.fit(X_train_count, y_train)

#aplico el transform del vectorizador fitteado en train
X_test = countvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

f1_3 = f1_score(y_test, y_pred, average='macro').item()
print(f1_3)

0.5121480984265602


In [21]:
#Entreno un modelo ComplementNB sobre el dataset de train vectorizado con TfidfVectorizer con los parámetros por default.

#fit del modelo sobre train
clf = ComplementNB()
clf.fit(X_train_tfidf, y_train)

#aplico el transform del vectorizador fitteado en train
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

#calculo el f1_score macro
f1_4 = f1_score(y_test, y_pred, average='macro').item()
print(f1_4)

0.692953349950875


In [22]:
#Entreno un modelo ComplementNB sobre el dataset de train vectorizado con TfidfVectorizer con min_df = 2.

#fit del modelo sobre train
clf = ComplementNB()
clf.fit(X_train_tfidf_m, y_train)

#aplico el transform del vectorizador fitteado en train
X_test = tfidfvect_m.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

#calculo el f1_score macro
f1_5 = f1_score(y_test, y_pred, average='macro').item()
print(f1_5)

0.6934824308370665


In [23]:
#Entreno un modelo ComplementNB sobre el dataset de train vectorizado con CountVectorizer

#fit del modelo sobre train
clf = ComplementNB()
clf.fit(X_train_count, y_train)

#aplico el transform del vectorizador fitteado en train
X_test = countvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

#calculo el f1_score macro
f1_6 = f1_score(y_test, y_pred, average='macro').item()
print(f1_6)

0.637426504468489


In [24]:
#Resumen de los resultados obtenidos
data = {
    "Vectorizer": [
        "TfidfVectorizer",
        "TfidfVectorizer (min_df=2)",
        "CountVectorizer"
    ],
    "MultinomialNB": [
        round(f1_1, 4),
        round(f1_2, 4),
        round(f1_3, 4)
    ],
    "ComplementNB": [
        round(f1_4, 4),
        round(f1_5, 4),
        round(f1_6, 4)
    ]
}

resumen_f1_score = pd.DataFrame(data)
resumen_f1_score

,Vectorizer,MultinomialNB,ComplementNB
0,TfidfVectorizer,0.5854,0.6930
1,TfidfVectorizer (min_df=2),0.5970,0.6935
2,CountVectorizer,0.5121,0.6374


Sobre el modelo que alcanzó el mejor f1_score macro, modifiqué el hiperparámetro alpha.
Este hiperparámetro es el valor que se asigna para los términos que no se encuentran dentro del vocabulario, y por default este valor es igual a 1. Me parecía interesante ver el impacto de darle un valor menor a 1 a estos términos.


In [25]:

clf = ComplementNB(alpha=0.5)
clf.fit(X_train_tfidf_m, y_train)

#aplico el transform del vectorizador fitteado en train
X_test = tfidfvect_m.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

#calculo el f1_score macro
f1_score(y_test, y_pred, average='macro').item()

0.6980382854233241

El mayor f1_score macro (0.6980) se obtuvo entrando un modelo ComplementNB con un alpha de 0.5,sobre un dataset vectorizado con TfidfVectorizer con el hiperparámetro min_df igual a 2.


**Punto 3**

In [26]:
# transpongo la matriz documento-termino para tener en las filas los vectores asociados a cada término.
X_train_tfidf_m_transposed = X_train_tfidf_m.transpose()

In [27]:
column_names = tfidfvect_m.get_feature_names_out()
column_names

array(['00', '000', '0000', ..., 'zyra', 'zyxel', 'zz'], dtype=object)

In [28]:
check_word_in_vocabulary = [x for x in enumerate(column_names) if 'car' in x]
check_word_in_vocabulary

[(8366, 'car')]

In [29]:
idx = [8366]

cossim = cosine_similarity(X_train_tfidf_m_transposed[idx], X_train_tfidf_m_transposed)[0]

idx2word = {v: k for k,v in tfidfvect_m.vocabulary_.items()}

# veo el término original y los 5 más similares:
mostsim = np.argsort(cossim)[::-1][0:6]

for i in mostsim:
  print('idx:',i, 'word:', idx2word[i])


idx: 8366 word: car
idx: 8445 word: cars
idx: 9157 word: civic
idx: 11381 word: dealer
idx: 26193 word: owner
idx: 21713 word: loan


In [30]:
check_word_in_vocabulary = [x for x in enumerate(column_names) if 'man' in x]
check_word_in_vocabulary

[(22396, 'man')]

In [31]:
idx = [22396]

cossim = cosine_similarity(X_train_tfidf_m_transposed[idx], X_train_tfidf_m_transposed)[0]

idx2word = {v: k for k,v in tfidfvect_m.vocabulary_.items()}

# veo el término original y los 5 más similares:
mostsim = np.argsort(cossim)[::-1][0:6]

for i in mostsim:
  print('idx:',i, 'word:', idx2word[i])


idx: 22396 word: man
idx: 31908 word: shintoism
idx: 10022 word: confucianism
idx: 39350 word: zoerasterism
idx: 28341 word: prupose
idx: 22391 word: mammal


In [32]:
check_word_in_vocabulary = [x for x in enumerate(column_names) if 'country' in x]
check_word_in_vocabulary

[(10538, 'country')]

In [33]:
idx = [10538]

cossim = cosine_similarity(X_train_tfidf_m_transposed[idx], X_train_tfidf_m_transposed)[0]

idx2word = {v: k for k,v in tfidfvect_m.vocabulary_.items()}

# veo el término original y los 5 más similares:
mostsim = np.argsort(cossim)[::-1][0:6]

for i in mostsim:
  print('idx:',i, 'word:', idx2word[i])

idx: 10538 word: country
idx: 7426 word: borden
idx: 8715 word: cess
idx: 29553 word: regan
idx: 6592 word: bastards
idx: 975 word: 1993apr24


In [34]:
check_word_in_vocabulary = [x for x in enumerate(column_names) if 'god' in x]
check_word_in_vocabulary

[(16573, 'god')]

In [35]:
idx = [16573]

cossim = cosine_similarity(X_train_tfidf_m_transposed[idx], X_train_tfidf_m_transposed)[0]

idx2word = {v: k for k,v in tfidfvect_m.vocabulary_.items()}

# veo el término original y los 5 más similares:
mostsim = np.argsort(cossim)[::-1][0:6]

for i in mostsim:
  print('idx:',i, 'word:', idx2word[i])

idx: 16573 word: god
idx: 19974 word: jesus
idx: 6987 word: bible
idx: 34847 word: that
idx: 14440 word: existence
idx: 9018 word: christ


In [36]:
check_word_in_vocabulary = [x for x in enumerate(column_names) if 'hardware' in x]
check_word_in_vocabulary

[(17313, 'hardware')]

In [37]:
idx = [17313]

cossim = cosine_similarity(X_train_tfidf_m_transposed[idx], X_train_tfidf_m_transposed)[0]

idx2word = {v: k for k,v in tfidfvect_m.vocabulary_.items()}

# veo el término original y los 5 más similares:
mostsim = np.argsort(cossim)[::-1][0:6]

for i in mostsim:
  print('idx:',i, 'word:', idx2word[i])

idx: 17313 word: hardware
idx: 36157 word: unconditionally
idx: 17250 word: handshaking
idx: 32633 word: software
idx: 31362 word: scrolloff
idx: 1350 word: 24xmode



Los resultados obtenidos al transponer la matriz documento-término y calcular la similitud coseno entre los términos muestran que las similitudes halladas no se fundamentan exclusivamente en sinónimos o palabras afines, sino en la coocurrencia de términos dentro de un mismo documento. Sin embargo existen casos en que los términos similares en su mayoría son palabras afines (ejemplo 'cars'), pero no se ve esto en general para el resto de los términos. Por ejemplo, los términos relacionados con 'god' tienden a concentrarse en el ámbito de la religión.